In [ ]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 8.3 MB/s 
     |████████████████████████████████| 10.2 MB 16.5 MB/s 
     |████████████████████████████████| 79 kB 1.2 MB/s 
     |████████████████████████████████| 140 kB 17.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
pip install dynamodb-json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 6.8 MB/s 


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 7.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13


In [ ]:
import boto3
import time 
import uuid 
from datetime import datetime  
from dynamodb_json import json_util as json 
from __future__ import print_function
import json
import decimal
import timeit
import requests
from google.colab import drive
from boto3.dynamodb.conditions import Key, Attr

In [ ]:
AWS_ACCESS = ""
AWS_SECRET = ""
AWS_REGION = "us-east-2"
TABLE_NAME = "movies"

In [ ]:
dynamodb_client = boto3.client('dynamodb',aws_secret_access_key=AWS_SECRET,aws_access_key_id=AWS_ACCESS,region_name=AWS_REGION)

In [ ]:
#dynamodb = client.resource('dynamodb', region_name='us-east-2')
#didnt work I think
movies_table = dynamodb_client.create_table(TableName='movies',KeySchema=[{'AttributeName': 'movieId','KeyType': 'HASH'},{'AttributeName': 'userId','KeyType': 'RANGE'}],AttributeDefinitions=[{'AttributeName': 'movieId','AttributeType': 'N'},{'AttributeName': 'userId','AttributeType': 'N'},],ProvisionedThroughput={'ReadCapacityUnits': 10,'WriteCapacityUnits': 10 })


In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
movies="/content/drive/My Drive/movierating.json"
print(movies)

/content/drive/My Drive/movierating.json


In [ ]:
#Inserting bulk data into the table
count=0
dynamodb = boto3.resource('dynamodb', region_name='us-east-2',aws_secret_access_key=AWS_SECRET,aws_access_key_id=AWS_ACCESS)
table = dynamodb.Table('movies')

In [ ]:
records=""
with open(movies, 'r') as datafile:
  records=json.load(datafile, parse_float = decimal.Decimal)

In [ ]:
count=0
for i in records:
  response=table.put_item(Item=i)
  count+=1
  if count==500:
    break


In [ ]:
#Putting Single Item into the table using PutItem
start = time.time()
mId = 5000
uId = 5
rating = 3
TS=1163373699

response = table.put_item(
   Item={
        'movieId': mId,
        'userId': uId,
        'rating': rating,
        'timestamp': TS
        }
)

end = time.time()
print(end - start)
print(response)

0.3102452754974365
{'ResponseMetadata': {'RequestId': 'CTQPCQMQGO95P8QG0GCVN25JCRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Dec 2022 17:26:31 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'CTQPCQMQGO95P8QG0GCVN25JCRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [ ]:
#scan
start = time.time()

response = table.scan()
data = response['Items']

while 'movieId' in response:
    response = table.scan(ExclusiveStartKey=response['movieId'])
    data.extend(response['Items'])

print(response)

end = time.time()
print(end - start)

{'Items': [{'movieId': Decimal('4666'), 'rating': Decimal('1'), 'userId': Decimal('15'), 'timestamp': Decimal('997937539')}, {'movieId': Decimal('107978'), 'rating': Decimal('5'), 'userId': Decimal('56'), 'timestamp': Decimal('1467003921')}, {'movieId': Decimal('27904'), 'rating': Decimal('2.5'), 'userId': Decimal('15'), 'timestamp': Decimal('1347936774')}, {'movieId': Decimal('2289'), 'rating': Decimal('5'), 'userId': Decimal('4'), 'timestamp': Decimal('949919556')}, {'movieId': Decimal('2289'), 'rating': Decimal('4'), 'userId': Decimal('15'), 'timestamp': Decimal('997938322')}, {'movieId': Decimal('2289'), 'rating': Decimal('4'), 'userId': Decimal('23'), 'timestamp': Decimal('1148671976')}, {'movieId': Decimal('2289'), 'rating': Decimal('5'), 'userId': Decimal('30'), 'timestamp': Decimal('945114406')}, {'movieId': Decimal('2289'), 'rating': Decimal('5'), 'userId': Decimal('34'), 'timestamp': Decimal('973746929')}, {'movieId': Decimal('2289'), 'rating': Decimal('3.5'), 'userId': Decim

In [ ]:
#Getting Single Item from the table using GetItem
start = time.time()
mId = 2502
uId = 6


try:
    response = table.get_item(
        Key={
            'movieId': mId,
            'userId': uId
        }
    )

except (ClientError, KeyError) as e:
        print_(e)

print(response)
end = time.time()
print(end - start)

{'Item': {'movieId': Decimal('2502'), 'rating': Decimal('3.5'), 'userId': Decimal('6'), 'timestamp': Decimal('1108134291')}, 'ResponseMetadata': {'RequestId': '3SH2SIDV8IQ33ED3S5GREMHSIJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Dec 2022 17:38:39 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '104', 'connection': 'keep-alive', 'x-amzn-requestid': '3SH2SIDV8IQ33ED3S5GREMHSIJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '549118328'}, 'RetryAttempts': 0}}
0.2999868392944336


In [ ]:
start = time.time()
response = table.query(
    KeyConditionExpression=Key('movieId').eq(2529)
)
print(response)
for i in response['Items']:
    print(i['movieId'], ":", i['userId'])
end = time.time()
print(end - start)

{'Items': [{'movieId': Decimal('2529'), 'rating': Decimal('4'), 'userId': Decimal('6'), 'timestamp': Decimal('1108134293')}, {'movieId': Decimal('2529'), 'rating': Decimal('1'), 'userId': Decimal('12'), 'timestamp': Decimal('968045475')}, {'movieId': Decimal('2529'), 'rating': Decimal('4'), 'userId': Decimal('30'), 'timestamp': Decimal('945295703')}, {'movieId': Decimal('2529'), 'rating': Decimal('4'), 'userId': Decimal('34'), 'timestamp': Decimal('973747643')}, {'movieId': Decimal('2529'), 'rating': Decimal('4'), 'userId': Decimal('41'), 'timestamp': Decimal('1093887555')}], 'Count': 5, 'ScannedCount': 5, 'ResponseMetadata': {'RequestId': '0CFHT9IA8VOK2GLVSLFED8RR8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Dec 2022 17:40:34 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '509', 'connection': 'keep-alive', 'x-amzn-requestid': '0CFHT9IA8VOK2GLVSLFED8RR8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '33992

In [ ]:
#filter
start = time.time()
response = table.scan(FilterExpression=Attr('userId').contains(6))

print(response)
end = time.time()
print(end - start)

{'Items': [], 'Count': 0, 'ScannedCount': 10375, 'ResponseMetadata': {'RequestId': 'SNQTE5MKU1UIDMRMP6Q0TMPJ8VVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Dec 2022 17:46:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '43', 'connection': 'keep-alive', 'x-amzn-requestid': 'SNQTE5MKU1UIDMRMP6Q0TMPJ8VVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3277706802'}, 'RetryAttempts': 0}}
0.1343064308166504


In [ ]:
#filter
start = time.time()
response = table.scan(FilterExpression=Attr('timestamp').gte(1109258257))
print(response)
for i in response['Items']:
    print(i['movieId'], ":", i['userId'])
end = time.time()
print(end - start)

Streaming output truncated to the last 5000 lines.
4645 : 17
7360 : 22
7360 : 48
5785 : 56
2174 : 22
2174 : 23
2174 : 60
2174 : 61
2174 : 68
440 : 5
102445 : 15
102445 : 48
102445 : 72
74851 : 15
6944 : 15
6944 : 23
954 : 23
47721 : 20
2080 : 61
6893 : 22
2054 : 61
2054 : 69
4359 : 23
8019 : 23
2126 : 15
2126 : 23
1909 : 22
3248 : 23
2469 : 68
5747 : 20
127108 : 56
2318 : 3
2318 : 48
69 : 26
69 : 73
471 : 15
471 : 23
471 : 73
3911 : 31
3911 : 59
3408 : 5
3408 : 17
3408 : 22
3408 : 23
1252 : 17
1252 : 23
1252 : 56
85510 : 48
112421 : 56
48322 : 56
1127 : 48
95510 : 15
95510 : 48
95510 : 56
1010 : 61
56782 : 15
56782 : 26
56782 : 48
56782 : 56
56782 : 72
1994 : 15
86884 : 56
90439 : 15
5810 : 23
73023 : 56
410 : 59
410 : 69
410 : 73
110 : 3
110 : 8
110 : 13
110 : 20
110 : 23
110 : 38
110 : 42
110 : 48
110 : 72
110 : 73
2065 : 29
288 : 17
288 : 48
288 : 73
102993 : 56
2011 : 17
2011 : 22
2011 : 23
2011 : 56
2011 : 69
5570 : 48
100272 : 56
86190 : 15
46972 : 15
46972 : 61
461 : 52
5096 : 2